In [9]:
import numpy as np
import pandas as pd
import zipfile
import csv


In [19]:
df = pd.read_csv('data/olid-training-v1.0.tsv', sep='\t')
df.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [24]:
X = df.drop(columns=['subtask_a', 'subtask_b','subtask_c'])
y = df.drop(columns=['id', 'tweet'])